In [11]:
import pandas as pd
import os
import logging

In [12]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [13]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [14]:
baojiadan_file_list = list()

In [15]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文报价单'):
        baojiadan_file_list.append(file_name)

In [16]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [17]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [18]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [19]:
wanted_shipping_service_list = ['燕文航空挂号小包-普货',
                                '燕文航空挂号小包-特货',
                                '燕文航空易派小包-普货',
                                '燕文航空易派小包-特货',
                                '燕文专线优先小包',
                                '燕文专线追踪小包-普货',
                                '燕文专线追踪小包-特货',
                                '燕邮宝挂号',
                                '燕文英国快线',
                                '中英YODEL专线-标准',
                                '中英YODEL专线-小包(不含电)',
                                '中英YODEL专线-小包(含电)',
                                '中邮北京挂号小包',
                                '中邮北京线下E邮宝',
                                '中邮广州挂号小包',
                                '中邮深圳挂号小包',
                                '比利时挂号小包（上海）',
                                '比利时挂号小包（香港）',
                                '德邮香港挂号小包',
                                '德邮香港挂号小包-特货',
                                '韩国epacket小包-深圳',
                                '荷兰邮政e-packet小包-特货',
                                '荷兰邮政挂号小包(不含电)',
                                '荷兰邮政挂号小包(含电)',
                                '荷兰邮政挂号小包(配电)',
                                '马来西亚挂号小包-深圳',
                                '瑞士邮政挂号小包(不含电)',
                                '瑞士邮政挂号小包(含电)',
                                '瑞邮派小包',
                                '土耳其邮政挂号小包(不含电)',
                                '土耳其邮政挂号小包(含电)',
                                '香港邮政挂号小包',
                                '新加坡邮政挂号小包',
                                '燕文C挂号小包',
                                '燕文C优先小包']

In [20]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
                        # print(country_cell_location)
                        break
            df = df[country_cell_location[0]:]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
            
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except:
            logging.info(service_name + ' failed to export.')
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2019-09-05 21:32:15,405 - INFO - 燕文航空挂号小包-普货 is ready to export.
 2019-09-05 21:32:15,435 - INFO - 燕文航空挂号小包-普货 exported.
 2019-09-05 21:32:15,436 - INFO - 燕文航空挂号小包-特货 is ready to export.
 2019-09-05 21:32:15,450 - INFO - 燕文航空挂号小包-特货 exported.
 2019-09-05 21:32:15,450 - INFO - 燕文航空易派小包-普货 is ready to export.
 2019-09-05 21:32:15,459 - INFO - 燕文航空易派小包-普货 exported.
 2019-09-05 21:32:15,460 - INFO - 燕文航空易派小包-特货 is ready to export.
 2019-09-05 21:32:15,467 - INFO - 燕文航空易派小包-特货 exported.
 2019-09-05 21:32:15,468 - INFO - 燕文专线追踪小包-普货 is ready to export.
 2019-09-05 21:32:15,476 - INFO - 燕文专线追踪小包-普货 exported.
 2019-09-05 21:32:15,477 - INFO - 燕文专线追踪小包-特货 is ready to export.
 2019-09-05 21:32:15,486 - INFO - 燕文专线追踪小包-特货 exported.
 2019-09-05 21:32:15,486 - INFO - 燕邮宝挂号 is ready to export.
 2019-09-05 21:32:15,492 - INFO - 燕邮宝挂号 exported.
 2019-09-05 21:32:15,492 - INFO - 燕文英国快线 is ready to export.
 2019-09-05 21:32:15,496 - INFO - 燕文英国快线 exported.
 2019-09-05 21:32:15,497 - INFO - 中英YODEL专线-小包